# cGAN Generate Synthetic Data for Adult Dataset

CTGAN model is based on the GAN-based Deep Learning data synthesizer 

In [1]:
from implementation_functions import *

import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose

import pandas as pd
import numpy as np
from prince import FAMD #Factor analysis of mixed data
from aif360.metrics import BinaryLabelDatasetMetric
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
import skfuzzy as fuzz

In [2]:
data_type = "adult"
dataset_orig, privileged_groups, unprivileged_groups = aif_data(data_type, False)

In [3]:
print(type(dataset_orig))

<class 'aif360.datasets.adult_dataset.AdultDataset'>


In [4]:
data_converted = dataset_orig.convert_to_dataframe()

In [5]:
data = data_converted[0]

In [6]:
data.head(5)

,age,education-num,race,sex,capital-gain,capital-loss,hours-per-week,workclass=Federal-gov,workclass=Local-gov,workclass=Private,...,native-country=Puerto-Rico,native-country=Scotland,native-country=South,native-country=Taiwan,native-country=Thailand,native-country=Trinadad&Tobago,native-country=United-States,native-country=Vietnam,native-country=Yugoslavia,income-per-year
0,25.0,7.0,0.0,1.0,0.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,38.0,9.0,1.0,1.0,0.0,0.0,50.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,28.0,12.0,1.0,1.0,0.0,0.0,40.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,44.0,10.0,0.0,1.0,7688.0,0.0,40.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
5,34.0,6.0,1.0,1.0,0.0,0.0,30.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [7]:
ds_shape = data_converted[0].shape[0]
print(ds_shape)

45222


In [8]:
sens_attr = ['race', 'sex']
decision_label = 'income-per-year'
# Getting the baseline peformance results from the imbalanced dataset
fav_l = 1
unfav_l = 0

# Creating the snythetic sub-class label column and num-cat columns identification
orig_df, num_list, cat_list = preprocess(data, sens_attr, decision_label)

data['sub_labels'].value_counts()

6    18268
2    10428
7     8752
4     2720
0     2598
3     1455
5      787
1      214
Name: sub_labels, dtype: int64

In [9]:
import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

--- 3.5762786865234375e-05 seconds ---


Before proceeding to generate synthetic dataset we need to make sure for each class label (0, 1, 2, ...,7), there is an equal number of generated datasets ..

#for the adult dataset we need this number of dataset for each sub_label
samples_needed = {
    "0": 17402,
    "1": 19786,
    "2": 9572,
    "3": 18545,
    "4": 17280,
    "5": 19213,
    "6": 1732,
    "7": 11248,
    }

In [16]:
# EQUALIZE THE POSITIVE AND THE NEGATIVE CLASS SIZES (due to extreme # of negative cases)
from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(sampling_strategy='majority')
X = orig_df.loc[:, orig_df.columns != 'income-per-year']
y = orig_df.loc[:, orig_df.columns == 'income-per-year']
y=y.astype('int')
X_over, y_over = undersample.fit_resample(X, y)
new_df = pd.concat([X_over, pd.DataFrame(y_over)], axis=1)
orig_df = new_df
#new subgroup counts
print(data['sub_labels'].value_counts())

6    18268
2    10428
7     8752
4     2720
0     2598
3     1455
5      787
1      214
Name: sub_labels, dtype: int64


In [17]:
# Train-test split WITH stratification
X = data.loc[:, data.columns != decision_label]
y = data.loc[:, data.columns == decision_label].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                    shuffle=True,
                                                    stratify=X['sub_labels'])

In [18]:
keep_sub_l = X_train['sub_labels']

In [19]:
X_train_new = X_train.drop(['age', 'sex', 'sub_labels'], axis=1)

In [20]:
X_train_new['sub_labels'] = keep_sub_l

In [21]:
X_train_new['class_labels'] = y_train

In [22]:
X_train_new

,education-num,race,capital-gain,capital-loss,hours-per-week,workclass=Federal-gov,workclass=Local-gov,workclass=Private,workclass=Self-emp-inc,workclass=Self-emp-not-inc,...,native-country=Scotland,native-country=South,native-country=Taiwan,native-country=Thailand,native-country=Trinadad&Tobago,native-country=United-States,native-country=Vietnam,native-country=Yugoslavia,sub_labels,class_labels
39478,9.0,1,0.0,0.0,55.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,6,0
25195,10.0,1,0.0,0.0,55.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,7,1
24278,13.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,2,0
25171,9.0,1,0.0,0.0,50.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,6,0
44605,13.0,1,0.0,0.0,38.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12665,9.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,2,0
538,9.0,1,0.0,0.0,60.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,7,1
27302,16.0,1,0.0,0.0,57.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,7,1
11876,10.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,6,0


# Here we start the GAN work

The execution time for fitting the dataset is 113.7167 minutes--> 2 hrs 19 minutes

In [23]:
from sdv.tabular import CTGAN
model = CTGAN()

You can use this cell in case you changed the training set size, so you have to fit the newly splitted data and train the GAN on them, other wise just load the pickel file that contains the fitted dataset into the model, and generate new samples (either conditioanlly generated or just specific numbers of samples).

In [24]:
loaded = CTGAN.load('cGAN_Adult_V3.pkl')

Generating X_train, the following cell shows, generating the samples according to the sub_labels 

In [25]:
#This cell is reposnsible about specifying the number of samples needed to be generated using the GAN, according to the sub_labels
available_rows = {}
for row_count in range(8):
    available_rows[row_count] = X_train["sub_labels"].value_counts()[row_count]
        
target_rows = max(available_rows.values())
max_label = max(available_rows, key=available_rows.get)
print(target_rows)
print(max_label)

12787
6


In [26]:
#make sure the newly generated samples are data frames
main_df = pd.DataFrame()

In [27]:
#Start generating the new samples and add them to the main data frame by concatinating them
for key, value in available_rows.items():
    if int(key) != int(max_label):
        conditions = {
            "sub_labels" : int(key),
        }
        needed_rows = target_rows - value
        main_df = pd.concat([main_df, loaded.sample(needed_rows, conditions=conditions)])
        
print(len(main_df.index))

70641


In [28]:
main_df

,education-num,race,capital-gain,capital-loss,hours-per-week,workclass=Federal-gov,workclass=Local-gov,workclass=Private,workclass=Self-emp-inc,workclass=Self-emp-not-inc,...,native-country=Scotland,native-country=South,native-country=Taiwan,native-country=Thailand,native-country=Trinadad&Tobago,native-country=United-States,native-country=Vietnam,native-country=Yugoslavia,sub_labels,class_labels
0,13.0,1,107.0,0.0,41.0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,9.0,1,96.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,10.0,1,147.0,2.0,40.0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,9.0,0,58.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,9.0,1,160.0,0.0,40.0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6656,11.0,1,7.0,0.0,62.0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,7,1
6657,16.0,0,0.0,0.0,40.0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,7,0
6658,15.0,0,0.0,0.0,45.0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,7,0
6659,14.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,7,1


# Random Forest Classifer

In [29]:
# Type the desired classifier to train the classification models with model obj
RF= RandomForestClassifier()
baseline_stats, cm, ratio_table, preds = baseline_metrics(RF, X_train, X_test, 
                                                  y_train, y_test, sens_attr, 
                                                  fav_l, unfav_l)

In [30]:
#Ratio outputs using the Random Forests
print(ratio_table)

                       Base Ratio  Positive Ratio  Negative Ratio
Index                                                            
{'race': 0, 'sex': 0}    0.062136        0.002801        0.059335
{'race': 1, 'sex': 0}    0.262770        0.025356        0.237414
{'race': 0, 'sex': 1}    0.077541        0.016658        0.060883
{'race': 1, 'sex': 1}    0.597553        0.178964        0.418589


In [31]:
test_sublabels = X_test['sub_labels']
X_test_n = X_test.drop(['age', 'sex','sub_labels'], axis=1)
num_list, cat_list = type_lists(X_test_n)

Concatinaing the whole set, the newly generated samples and the traning set.

In [32]:
final_df = pd.concat([main_df, X_train_new])

In [33]:
final_df

,education-num,race,capital-gain,capital-loss,hours-per-week,workclass=Federal-gov,workclass=Local-gov,workclass=Private,workclass=Self-emp-inc,workclass=Self-emp-not-inc,...,native-country=Scotland,native-country=South,native-country=Taiwan,native-country=Thailand,native-country=Trinadad&Tobago,native-country=United-States,native-country=Vietnam,native-country=Yugoslavia,sub_labels,class_labels
0,13.0,1,107.0,0.0,41.0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,9.0,1,96.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
2,10.0,1,147.0,2.0,40.0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,9.0,0,58.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,9.0,1,160.0,0.0,40.0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12665,9.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,2,0
538,9.0,1,0.0,0.0,60.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,7,1
27302,16.0,1,0.0,0.0,57.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,7,1
11876,10.0,1,0.0,0.0,40.0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,6,0


In [34]:
final_df["sub_labels"].value_counts()

7    12787
6    12787
5    12787
4    12787
3    12787
2    12787
1    12787
0    12787
Name: sub_labels, dtype: int64

In [35]:
# Predicting the test sets using the random Forests
X_test_pred_RF= predict_whole_set(clf, final_df, X_test_n)

In [36]:
metrics_table1, cm1, ratio_t1 = metrics_calculate(X_test, X_test_pred_RF, y_test,
                                                  sens_attr, fav_l, unfav_l)


In [37]:
#outputs from the Random Forests
print(metrics_table1)
print("Confusion Matrix:", cm1)
print(ratio_t1)

                                                AEO Difference  \
[{'race': 0, 'sex': 0}][{'race': 1, 'sex': 0}]       -0.066799   
[{'race': 1, 'sex': 0}][{'race': 0, 'sex': 1}]       -0.076065   
[{'race': 0, 'sex': 1}][{'race': 1, 'sex': 1}]       -0.028300   
[{'race': 0, 'sex': 0}][{'race': 0, 'sex': 1}]       -0.142864   
[{'race': 1, 'sex': 0}][{'race': 1, 'sex': 1}]       -0.104365   
[{'race': 0, 'sex': 0}][{'race': 1, 'sex': 1}]       -0.171163   

                                                Disparate Impact Ratio  \
[{'race': 0, 'sex': 0}][{'race': 1, 'sex': 0}]                0.480842   
[{'race': 1, 'sex': 0}][{'race': 0, 'sex': 1}]                0.466434   
[{'race': 0, 'sex': 1}][{'race': 1, 'sex': 1}]                0.705893   
[{'race': 0, 'sex': 0}][{'race': 0, 'sex': 1}]                0.224281   
[{'race': 1, 'sex': 0}][{'race': 1, 'sex': 1}]                0.329253   
[{'race': 0, 'sex': 0}][{'race': 1, 'sex': 1}]                0.158319   

                  

# Extreme Gradient Boosting Classifier

In [38]:
xgboost= GradientBoostingClassifier()
baseline_stats, cm, ratio_table, preds = baseline_metrics(clf, X_train, X_test, 
                                                  y_train, y_test, sens_attr, 
                                                  fav_l, unfav_l)

In [42]:
#Ratio outputs using the XGBoost
print(ratio_table)

                       Base Ratio  Positive Ratio  Negative Ratio
Index                                                            
{'race': 0, 'sex': 0}    0.062136        0.002801        0.059335
{'race': 1, 'sex': 0}    0.262770        0.025872        0.236898
{'race': 0, 'sex': 1}    0.077541        0.016584        0.060957
{'race': 1, 'sex': 1}    0.597553        0.180069        0.417484


In [39]:
# Predicting the test sets using the XGBoost
X_test_pred_xgb= predict_whole_set(xgboost, final_df, X_test_n)

In [40]:
metrics_table1, cm1, ratio_t1 = metrics_calculate(X_test, X_test_pred_xgb, y_test,
                                                  sens_attr, fav_l, unfav_l)

In [41]:
#outputs from the Random Forests
print(metrics_table1)
print("Confusion Matrix:", cm1)
print(ratio_t1)

                                                AEO Difference  \
[{'race': 0, 'sex': 0}][{'race': 1, 'sex': 0}]       -0.053170   
[{'race': 1, 'sex': 0}][{'race': 0, 'sex': 1}]       -0.074396   
[{'race': 0, 'sex': 1}][{'race': 1, 'sex': 1}]       -0.039290   
[{'race': 0, 'sex': 0}][{'race': 0, 'sex': 1}]       -0.127566   
[{'race': 1, 'sex': 0}][{'race': 1, 'sex': 1}]       -0.113686   
[{'race': 0, 'sex': 0}][{'race': 1, 'sex': 1}]       -0.166856   

                                                Disparate Impact Ratio  \
[{'race': 0, 'sex': 0}][{'race': 1, 'sex': 0}]                0.543479   
[{'race': 1, 'sex': 0}][{'race': 0, 'sex': 1}]                0.399335   
[{'race': 0, 'sex': 1}][{'race': 1, 'sex': 1}]                0.662596   
[{'race': 0, 'sex': 0}][{'race': 0, 'sex': 1}]                0.217030   
[{'race': 1, 'sex': 0}][{'race': 1, 'sex': 1}]                0.264598   
[{'race': 0, 'sex': 0}][{'race': 1, 'sex': 1}]                0.143803   

                  